In [27]:
from pyspark.sql import SparkSession

In [28]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('test')\
  .getOrCreate()

### Green Taxi

In [29]:
df_green = spark.read.parquet('./data/pq/green/*/*')

In [30]:
df_green.createOrReplaceTempView('green_trip_data')

In [31]:
df_green_revenue = spark.sql("""
  SELECT 
    date_trunc("hour", lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(*) AS row_count
  FROM green_trip_data
  WHERE lpep_pickup_datetime >= '2020-01-01 00:00:00'
  GROUP BY 1, 2
  ORDER BY 1, 2
""")

In [32]:
df_green_revenue.show()

+-------------------+----+------------------+---------+
|               hour|zone|            amount|row_count|
+-------------------+----+------------------+---------+
|2020-01-01 00:00:00|   7| 769.7299999999997|       45|
|2020-01-01 00:00:00|  17|            195.03|        9|
|2020-01-01 00:00:00|  18|               7.8|        1|
|2020-01-01 00:00:00|  22|              15.8|        1|
|2020-01-01 00:00:00|  24|              87.6|        3|
|2020-01-01 00:00:00|  25|             531.0|       26|
|2020-01-01 00:00:00|  29|              61.3|        1|
|2020-01-01 00:00:00|  32| 68.94999999999999|        2|
|2020-01-01 00:00:00|  33|317.27000000000004|       11|
|2020-01-01 00:00:00|  35|129.95999999999998|        5|
|2020-01-01 00:00:00|  36|295.34000000000003|       11|
|2020-01-01 00:00:00|  37|175.67000000000002|        6|
|2020-01-01 00:00:00|  38| 98.78999999999999|        2|
|2020-01-01 00:00:00|  40|168.98000000000002|        8|
|2020-01-01 00:00:00|  41|1363.9599999999987|   

In [33]:
df_green_revenue\
  .repartition(20)\
  .write.parquet('data/report/revenue/green', mode='overwrite')

25/03/14 11:19:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/14 11:19:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### Yellow Taxi

In [34]:
df_yellow = spark.read.parquet('./data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow_trip_data')

In [35]:
df_yellow_revenue = spark.sql("""
  SELECT 
    date_trunc("hour", tpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(*) AS row_count
  FROM yellow_trip_data
  WHERE tpep_pickup_datetime >= '2020-01-01 00:00:00'
  GROUP BY 1, 2
  ORDER BY 1, 2
""")

In [36]:
df_yellow_revenue\
  .repartition(20)\
  .write.parquet('data/report/revenue/yellow', mode='overwrite')

25/03/14 11:19:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/14 11:19:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


## JOINS
### External Merge Sort (Sort Merge Join)

In [37]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [38]:
df_green_revenue_tmp = df_green_revenue\
  .withColumnRenamed('amount', 'green_amount')\
  .withColumnRenamed('row_count', 'green_row_count')

In [39]:
df_yellow_revenue_tmp = df_yellow_revenue\
  .withColumnRenamed('amount', 'yellow_amount')\
  .withColumnRenamed('row_count', 'yellow_row_count')

In [40]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [41]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

25/03/14 11:19:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### Broadcast Join

In [42]:
df_zones = spark.read\
  .option("header", True)\
  .csv('data/raw/zones/')

In [43]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [ ]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID).drop('LocationID', 'zone')

In [45]:
df_result.show()

+-------------------+----+------------------+---------------+------------------+----------------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_row_count|     yellow_amount|yellow_row_count|  Borough|                Zone|service_zone|
+-------------------+----+------------------+---------------+------------------+----------------+---------+--------------------+------------+
|2020-01-01 00:00:00|  22|              15.8|              1|              NULL|            NULL| Brooklyn|    Bensonhurst West|   Boro Zone|
|2020-01-01 00:00:00|  25|             531.0|             26|            324.35|              16| Brooklyn|         Boerum Hill|   Boro Zone|
|2020-01-01 00:00:00|  55|129.29000000000002|              4|              NULL|            NULL| Brooklyn|        Coney Island|   Boro Zone|
|2020-01-01 00:00:00|  56|             99.69|              3|              18.1|               2|   Queens|              Corona|   Boro Zone|
|2020-

In [ ]:
df_result\
  .write.parquet('data/report/revenue/total_zones', mode='overwrite')

25/03/14 11:20:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [ ]:
# spark.stop()